# Kapitel 7: Bygga chattapplikationer
## OpenAI API Snabbstart

Denna notebook är anpassad från [Azure OpenAI Samples Repository](https://github.com/Azure/azure-openai-samples?WT.mc_id=academic-105485-koreyst) som innehåller notebooks som använder [Azure OpenAI](notebook-azure-openai.ipynb)-tjänster.

Python OpenAI API fungerar även med Azure OpenAI-modeller, med några få justeringar. Läs mer om skillnaderna här: [Hur du växlar mellan OpenAI och Azure OpenAI endpoints med Python](https://learn.microsoft.com/azure/ai-services/openai/how-to/switching-endpoints?WT.mc_id=academic-109527-jasmineg)


# Översikt  
"Stora språkmodeller är funktioner som omvandlar text till text. Givet en inmatad textsträng försöker en stor språkmodell förutsäga vilken text som kommer härnäst"(1). Denna "snabbstartsguide" kommer att introducera användare till grundläggande LLM-koncept, centrala paketkrav för att komma igång med AML, en mjuk introduktion till promptdesign samt flera korta exempel på olika användningsområden.


## Innehållsförteckning  

[Översikt](../../../../07-building-chat-applications/python)  
[Hur du använder OpenAI-tjänsten](../../../../07-building-chat-applications/python)  
[1. Skapa din OpenAI-tjänst](../../../../07-building-chat-applications/python)  
[2. Installation](../../../../07-building-chat-applications/python)    
[3. Autentiseringsuppgifter](../../../../07-building-chat-applications/python)  

[Användningsområden](../../../../07-building-chat-applications/python)    
[1. Sammanfatta text](../../../../07-building-chat-applications/python)  
[2. Klassificera text](../../../../07-building-chat-applications/python)  
[3. Generera nya produktnamn](../../../../07-building-chat-applications/python)  
[4. Finjustera en klassificerare](../../../../07-building-chat-applications/python)  

[Referenser](../../../../07-building-chat-applications/python)


### Skapa din första prompt  
Den här korta övningen ger en grundläggande introduktion till hur du skickar prompts till en OpenAI-modell för en enkel uppgift: "sammanfattning".


**Steg**:  
1. Installera OpenAI-biblioteket i din python-miljö  
2. Ladda in vanliga hjälpbibliotek och ange dina vanliga OpenAI-säkerhetsuppgifter för den OpenAI-tjänst du har skapat  
3. Välj en modell för din uppgift  
4. Skapa en enkel prompt till modellen  
5. Skicka din förfrågan till modellens API!


### 1. Installera OpenAI


In [ ]:
%pip install openai python-dotenv

In [ ]:
import os
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()

API_KEY = os.getenv("OPENAI_API_KEY","")
assert API_KEY, "ERROR: OpenAI Key is missing"

client = OpenAI(
    api_key=API_KEY
    )


### 3. Hitta rätt modell  
GPT-3.5-turbo eller GPT-4-modellerna kan förstå och generera naturligt språk.


In [ ]:
# Select the General Purpose curie model for text
model = "gpt-3.5-turbo"

## 4. Utformning av promptar  

"Magin med stora språkmodeller är att de, genom att tränas på att minimera detta prediktionsfel över enorma mängder text, till slut lär sig begrepp som är användbara för dessa förutsägelser. Till exempel lär de sig begrepp som"(1):

* hur man stavar
* hur grammatik fungerar
* hur man omformulerar
* hur man besvarar frågor
* hur man för en konversation
* hur man skriver på många språk
* hur man programmerar
* osv.

#### Hur man styr en stor språkmodell  
"Av alla indata till en stor språkmodell är textprompten utan tvekan den mest inflytelserika(1).

Stora språkmodeller kan uppmanas att generera utdata på några olika sätt:

Instruktion: Tala om för modellen vad du vill ha
Komplettering: Få modellen att slutföra början på det du vill ha
Demonstration: Visa modellen vad du vill ha, antingen med:
Några exempel i prompten
Många hundra eller tusentals exempel i en finjusterings-datamängd"



#### Det finns tre grundläggande riktlinjer för att skapa promptar:

**Visa och berätta**. Gör det tydligt vad du vill ha, antingen genom instruktioner, exempel eller en kombination av båda. Om du vill att modellen ska rangordna en lista med objekt i alfabetisk ordning eller klassificera ett stycke efter känsla, visa att det är det du vill ha.

**Ge kvalitetsdata**. Om du försöker bygga en klassificerare eller få modellen att följa ett mönster, se till att det finns tillräckligt med exempel. Korrekturläs dina exempel — modellen är oftast tillräckligt smart för att se igenom enkla stavfel och ge dig ett svar, men den kan också anta att det är avsiktligt och det kan påverka svaret.

**Kontrollera dina inställningar.** Temperatur- och top_p-inställningarna styr hur deterministisk modellen är när den genererar ett svar. Om du ber om ett svar där det bara finns ett rätt svar, vill du sätta dessa lägre. Om du vill ha mer varierade svar kan du sätta dem högre. Det vanligaste misstaget folk gör med dessa inställningar är att tro att de styr "smarthet" eller "kreativitet".


Källa: https://learn.microsoft.com/azure/ai-services/openai/overview


In [ ]:
# Create your first prompt
text_prompt = "Should oxford commas always be used?"

response = client.chat.completions.create(
  model=model,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":text_prompt},])

response.choices[0].message.content

In [ ]:

response = client.chat.completions.create(
  model=model,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":text_prompt},])

response.choices[0].message.content

## Sammanfatta text  
#### Utmaning  
Sammanfatta text genom att lägga till ett 'tl;dr:' i slutet av en textpassage. Lägg märke till hur modellen förstår att utföra flera uppgifter utan extra instruktioner. Du kan experimentera med mer beskrivande uppmaningar än tl;dr för att ändra modellens beteende och anpassa den sammanfattning du får(3).  

Nyligen har forskning visat betydande framsteg inom många NLP-uppgifter och benchmarktester genom att förträna på en stor textkorpus och sedan finjustera på en specifik uppgift. Även om denna metod vanligtvis är uppgiftsoberoende i sin arkitektur, kräver den ändå uppgiftsspecifika finjusteringsdatamängder med tusentals eller tiotusentals exempel. Människor kan däremot oftast utföra en ny språkuppgift med bara några få exempel eller enkla instruktioner – något som nuvarande NLP-system fortfarande har svårt med. Här visar vi att större språkmodeller kraftigt förbättrar uppgiftsoberoende, få-exempel-prestanda, ibland till och med i nivå med tidigare toppmoderna finjusteringsmetoder.  

Tl;dr


# Övningar för flera användningsområden  
1. Sammanfatta text  
2. Kategorisera text  
3. Skapa nya produktnamn


In [ ]:
prompt = "Recent work has demonstrated substantial gains on many NLP tasks and benchmarks by pre-training on a large corpus of text followed by fine-tuning on a specific task. While typically task-agnostic in architecture, this method still requires task-specific fine-tuning datasets of thousands or tens of thousands of examples. By contrast, humans can generally perform a new language task from only a few examples or from simple instructions - something that current NLP systems still largely struggle to do. Here we show that scaling up language models greatly improves task-agnostic, few-shot performance, sometimes even reaching competitiveness with prior state-of-the-art fine-tuning approaches.\n\nTl;dr"


In [ ]:
#Setting a few additional, typical parameters during API Call

response = client.chat.completions.create(
  model=model,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":prompt},])

response.choices[0].message.content

## Klassificera text  
#### Utmaning  
Klassificera objekt i kategorier som anges vid inferenstillfället. I följande exempel anger vi både kategorierna och texten som ska klassificeras i prompten (*playground_reference).

Kundförfrågan: Hej, en av tangenterna på mitt laptop-tangentbord gick sönder nyligen och jag behöver en ersättning:

Klassificerad kategori:


In [ ]:
prompt = "Classify the following inquiry into one of the following: categories: [Pricing, Hardware Support, Software Support]\n\ninquiry: Hello, one of the keys on my laptop keyboard broke recently and I'll need a replacement:\n\nClassified category:"
print(prompt)

In [ ]:
#Setting a few additional, typical parameters during API Call

response = client.chat.completions.create(
  model=model,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":prompt},])

response.choices[0].message.content

## Skapa nya produktnamn
#### Utmaning
Skapa produktnamn utifrån exempelord. Här inkluderar vi information om produkten vi ska generera namn för i prompten. Vi ger också ett liknande exempel för att visa det mönster vi vill ha. Vi har dessutom ställt in temperaturvärdet högt för att öka slumpmässigheten och få mer innovativa svar.

Produktbeskrivning: En milkshakemaskin för hemmet  
Fröord: snabb, hälsosam, kompakt.  
Produktnamn: HomeShaker, Fit Shaker, QuickShake, Shake Maker

Produktbeskrivning: Ett par skor som passar alla fotstorlekar.  
Fröord: anpassningsbar, passform, omni-fit.


In [ ]:
prompt = "Product description: A home milkshake maker\nSeed words: fast, healthy, compact.\nProduct names: HomeShaker, Fit Shaker, QuickShake, Shake Maker\n\nProduct description: A pair of shoes that can fit any foot size.\nSeed words: adaptable, fit, omni-fit."

print(prompt)

In [ ]:
#Setting a few additional, typical parameters during API Call

response = client.chat.completions.create(
  model=model,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":prompt}])

response.choices[0].message.content

# Referenser  
- [Openai Cookbook](https://github.com/openai/openai-cookbook?WT.mc_id=academic-105485-koreyst)  
- [OpenAI Studio Exempel](https://oai.azure.com/portal?WT.mc_id=academic-105485-koreyst)  
- [Bästa praxis för finjustering av GPT-3 för att klassificera text](https://docs.google.com/document/d/1rqj7dkuvl7Byd5KQPUJRxc19BJt8wo0yHNwK84KfU3Q/edit#?WT.mc_id=academic-105485-koreyst)


# För mer hjälp  
[OpenAI Commercialization Team](AzureOpenAITeam@microsoft.com)


# Bidragsgivare
* Louis Li



---

**Ansvarsfriskrivning**:  
Detta dokument har översatts med hjälp av AI-översättningstjänsten [Co-op Translator](https://github.com/Azure/co-op-translator). Vi strävar efter noggrannhet, men var medveten om att automatiska översättningar kan innehålla fel eller brister. Det ursprungliga dokumentet på dess originalspråk ska betraktas som den auktoritativa källan. För kritisk information rekommenderas professionell mänsklig översättning. Vi tar inget ansvar för eventuella missförstånd eller feltolkningar som uppstår vid användning av denna översättning.
